In [1]:
import pandas as pd

# JSQuAD

In [2]:
import json
import pandas as pd
from urllib.request import urlopen
import json

# download and place jsquad train file
# https://github.com/yahoojapan/JGLUE/blob/main/datasets/jsquad-v1.1/train-v1.1.json
with open("./train-v1.1.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

data = []
for d in json_data["data"]:
    title = d["title"]
    for p in d["paragraphs"]:
        context = p["context"]
        for qa in p["qas"]:
            data.append([
                title,
                context,
                qa["question"],
                qa["answers"][0]["text"],
            ])
df_jsquad = pd.DataFrame(data, columns=["title", "context", "question", "answer"])
df_jsquad.shape

(62859, 4)

In [3]:
# contextとanswerが共通しているデータを削除
df_jsquad = df_jsquad.drop_duplicates(subset=["context", "answer"], keep="first")
df_jsquad.shape

(46641, 4)

# JaQuAD

In [ ]:
from datasets import load_dataset
jaquad_data = load_dataset('SkelterLabsInc/JaQuAD')

In [5]:
jaquad_data["train"][0]

{'id': 'tr-000-00-000',
 'title': '手塚治虫',
 'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
 'question': '戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?',
 'question_type': 'Multiple sentence reasoning',
 'answers': {'text': ['手塚治虫'], 'answer_start': [0], 'answer_type': ['Person']}}

In [6]:
df_jqauad = pd.DataFrame(
    [[e["title"], e["context"], e["question"], e["answers"]["text"][0]] for e in jaquad_data["train"]],
    columns=["title", "context", "question", "answer"]
    )

In [8]:
df = pd.concat([df_jqauad, df_jsquad])
df.shape

(78389, 4)

# Prompts

In [9]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_str}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

# Training Pattern

In [ ]:
IMPOSSIBLE_TARGET = "すみません、答えられません。"

def create_default(row, df):
    prompt = DEFAULT_PROMPT.format(
        context_str=row["context"],
        query_str=row["question"]
        )
    target = row["answer"]
    return prompt, target, "default"

def create_impossible(row, df):
    # ランダムに、異なるtitleのcontextに対する質問をピックアップする
    context = row["context"]
    impossible_question = df[df["title"]!=row["title"]].sample(1).iloc[0]["question"]
    prompt = DEFAULT_PROMPT.format(
        context_str=context,
        query_str=impossible_question
        )
    return prompt, IMPOSSIBLE_TARGET, "impossible"

import random
def create_refine_incorrect2correct(row, df):
    # 誤った答えを正しいcontextでrefineする
    context = row["context"]
    wrong_answer = df[df["title"]!=row["title"]].sample(1).iloc[0]["answer"]
    wrong_answer = random.choice([wrong_answer, IMPOSSIBLE_TARGET])
    prompt = REFINE_PROMPT.format(
        query_str=row["question"],
        existing_answer=wrong_answer,
        context_str=row["context"],
    )
    return prompt, row["answer"], "incorrect2correct"

def create_refine_incorrect2incorrect(row, df):
    # 誤った答えだが、contextが異なるためrefineしない
    # IMPOSSIBLE_TARGETにはせず、wrong_answerをそのまま返す
    wrong_context = df[df["title"]!=row["title"]].sample(1).iloc[0]["context"]
    wrong_answer = df[df["title"]!=row["title"]].sample(1).iloc[0]["answer"]
    wrong_answer = random.choice([wrong_answer, IMPOSSIBLE_TARGET])
    prompt = REFINE_PROMPT.format(
        query_str=row["question"],
        existing_answer=wrong_answer,
        context_str=wrong_context,
    )
    return prompt, wrong_answer, "incorrect2incorrect"

def create_refine_correct2incorrect(row, df):
    # 正しい答えを誤ったcontextでrefineしない
    wrong_context = df[df["title"]!=row["title"]].sample(1).iloc[0]["context"]
    prompt = REFINE_PROMPT.format(
        query_str=row["question"],
        existing_answer=row["answer"],
        context_str=wrong_context,
    )
    return prompt, row["answer"], "correct2incorrect"

def create_refine_correct2correct(row, df):
    # 正しい答えを正しいcontextでそのまま返す
    prompt = REFINE_PROMPT.format(
        query_str=row["question"],
        existing_answer=row["answer"],
        context_str=row["context"],
    )
    return prompt, row["answer"], "correct2correct"

In [10]:
import numpy as np 
# パターンごとの学習データの割合を決める
p = [
    2.0, 
    0.4, # モデルがimpossibleを返しすぎることを防ぐため学習件数を少なくする
    1.0,
    1.0,
    1.0,
    1.0,
    ]
pattern_list = np.random.choice(
    a=[
        create_default,
        create_impossible, 
        create_refine_incorrect2correct,
        create_refine_incorrect2incorrect,
        create_refine_correct2correct,
        create_refine_correct2incorrect,
        ],
    size=len(df),
    p=np.array(p)/sum(p)
    )

# データセット作成

In [11]:
import tqdm
train_data = []
for i in tqdm.tqdm(range(len(df))):
    func = pattern_list[i]
    row = df.iloc[i]
    prompt, target, category = func(row, df)
    train_data.append([prompt, target, category])
train_df = pd.DataFrame(train_data, columns=["prompt", "target", "category"])
train_df.shape

100%|██████████| 78389/78389 [10:11<00:00, 128.27it/s]


(78389, 3)

In [12]:
train_df.sample(1).iloc[0].values

array(['\n文脈情報は以下です。\n---\n橋は1977年より測量や設計、および用地買収に着手され、1984年夏より橋の施工が開始された。施工は日本橋梁、および三菱重工業(現、三菱重工鉄構エンジニアリング)が行い、深い谷でベント(仮受け台)の設置が困難な場所なため、架設工法としてケーブルクレーンによる斜吊り工法(ケーブルエレクション斜吊り工法とも呼ぶ)が用いられた。橋長149.200m、総幅員6.500m、有効幅員5.250m(車道3.750m、歩道1.500m)。最大支間長は99.000mで、支間割は21.700m、99.000m、21.700mである。なお左岸側でコンクリート橋に接続され、その橋を含めた総延長は159.6mと旧橋とほぼ同じ長さとなる。ただし、径間長やアーチライズ比が旧橋と異なるため、景観美が損なわれてしまった。1986年2月15日午前11時より橋の閉合式が開催され、荒川村長をはじめ地元および工事関係者が出席した。式典では神事のあと、関係者が見守る中ケーブルクレーンで搬送された7.5メートル重さ3.7トンの桁が両岸より伸びる橋桁の隙間に降下させた後結合して、橋が一本に繋がった。役目を終えたケーブルクレーンなどの仮設工は解体撤去され、床版コンクリート打設や橋桁塗装などの工事が行なわれ、橋は1986年8月完成し、同年9月3日開通した。開通式は午前11時より新橋袂の特設テントにて挙行され、地元選出の代議士や県会議員が来賓として招かれた。式典は祝辞の後、荒川村長らによるテープカットが執り行われ、この後に渡り初めを行う予定だったが、台風崩れの低気圧による大雨のため渡り初めは中止され、二組の三世代家族によるくす玉開披に変更となった。橋は同日15時から一般供用が開始された。\n---\n事前知識ではなく、文脈情報を参考に質問に答えてください。：橋の架設工法として用いられた工法は何ですか。\n',
       'ケーブルクレーンによる斜吊り工法', 'default'], dtype=object)

In [13]:
train_df["category"].value_counts()

default                24565
incorrect2correct      12262
correct2correct        12202
correct2incorrect      12201
incorrect2incorrect    12185
impossible              4974
Name: category, dtype: int64

In [14]:
train_df.to_csv("train.csv", index=False, encoding="utf-8")